# Fase 2: Extracción de Detalles de Películas con Selenium

WEBS
- IMDB: https://www.imdb.com/
- Rotten Tomatoes: https://www.rottentomatoes.com/

- Puntuacion de IMDB (en caso de que la tenga). ✅

- Puntuacion de Rotten Tomatoes (Tomatometer). ✅

- Dirección (director/a o directore/as de cada película). ✅

- Gionistas (de cada película). ✅

- Argumento. ✅

- Duración (en minutos). ✅

- Nombre de la película. ✅

1. Importamos librerías

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

#Imoprtamos librería para ver el % de avance
from tqdm import tqdm

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

2. Importamos csv

In [45]:
#Importamos los datos de la Fase 1
fase1=pd.read_csv('Fase1.csv')
fase1

,tipo_pelicula,titulo_pelicula,anno_estreno,mes_estreno,id_pelicula,genero_pelicula
0,Movie,Rodney Cecil: Psycho Hero,2011,4,tt1929361,Comedy
1,Short,Object of Desire,2023,9,tt29009897,Drama
2,Short,Making Magic,2015,NaN,tt3407398,Comedy
3,Short,Significant Other,2018,2,tt8042420,Drama
4,Movie,Hosea: A Story of Hope and Forgiveness,2015,2,tt4209582,Drama
...,...,...,...,...,...,...
995,Short,Article 32,2012,10,tt2439330,Drama
996,Movie,The Mongolian Dream,2012,3,tt1979369,Drama
997,Short,Pop!,2011,7,tt2650758,Comedy
998,Movie,Hakki,2024,4,tt30810790,Drama


3. Creamos zip, que utilizaremos en Rotten Tomatoes

In [46]:
# Extraemos la información de la columna 'Id_peli' y la guardamos en la variable 'lista_id'
lista_id = list(fase1['id_pelicula'])

#Extraemos la información de las columnas 'Nombre' y 'Anno_estreno' y las guardamos en las variables 'lista_pelis_cortos' y 'annos_pelis_cortos'
lista_nombre_pelis = list(fase1['titulo_pelicula'])
lista_anno_pelis = list(fase1['anno_estreno'])
#Unimos esas dos listas en un zip que utilizaremos para buscar dentro de Rotten Tomatoes
zip_pelis_cortos= list(zip(lista_nombre_pelis, lista_anno_pelis))

4. Creamos listas para DataFrame

In [47]:
# creamos listas para almacenar la información obtetina de las webs
lista_puntu_IMDB = []
lista_puntu_tomato = []
lista_dires = []
lista_guionista = []
lista_argumentos = []
lista_duraciones = []

5. Extraemos datos de IMDB

In [48]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in tqdm(lista_id):

    # Introducimos el número de ID de la peli en el buscador de la web
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(2)

    #Buscamos la puntuación de cada película
    try:
        #La puntuación está en un botón. Extraemos el texto de ese botón
        boton=driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1)').text
        #Extraemos la puntuación del texto
        parte_punt = boton.split('/')[0] #Esto devuelve un string con la estructura: 'PUNTUACIÓN EN IMDb\n6,6\n'
        #Extraemos la puntuación del string: una secuencia de dígitos seguidos de coma y dígitos
        patron_punt = r'(\d+,\d+)'
        puntuacion_str=re.search(patron_punt, parte_punt).group(0) #Esto devuelve un string '6,6', así que lo pasamos a float
        puntuacion_float=float(puntuacion_str.replace(',', '.'))
        #Appendeamos la puntuación en la lista
        lista_puntu_IMDB.append(puntuacion_float)

    except:
        lista_puntu_IMDB.append(None)


    #Buscamos la información de los directores de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i
    i=0
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más directores dé error y salganmos del loop while
        try:
            lista_dires.append(driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
        except:
            lista_dires.append(None)
            break


    #Buscamos la información de los guionistas de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i
    i=0
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más guionistas dé error y salganmos del loop while
        try:
            lista_guionista.append(driver.find_element('xpath', f'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
        except:
            lista_guionista.append(None)
            break


    #Buscamos el argumento de cada película
    #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
    try:
        lista_argumentos.append(driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text)
    except:          
        lista_argumentos.append(None)


    #Buscamos la duración de cada película en minutos 
    #Creamos lista en la que guardaremos los dígitos de las horas y los minutos
    horas_y_minutos=[]

    # Iniciamos try/expect loop para que busque la sección. Si la encuenta la appendea en seccion_duracion, de lo contrario, appendea None.
    try:
        duracion = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)').text
    except:
        duracion = 'NULL'
            
    #De la sección, nos devuelve una lista con todos los campos. Hacemos un regex para extraer las horas y los minutos.
    #Buscamos el patrón "Xh Ymin": un número seguido de 'h', seguido de 0 o más espacios, seguido de otro número y 'min'
    patron = r'(\d+)h\s*(\d+)min|(\d+)h\s*(\d+)m'
        
    #Dentro de cada sección, buscamos por texto, que es cada parte de la sección (título, año, duración)
    #Buscamos el patrón definido dentro de cada parte de la sección, y lo guardamos en la variable 'match'
    match=re.search(patron, duracion)
    if match:
        #Si ha encontrado el patrón, guarda el primer grupo de captura (\d+) en la variable 'horas' y el segundo (\d+) en la variable 'minutos'
        horas=match.group(1)
        minutos=match.group(2)
        horas_y_minutos.append((int(horas), int(minutos)))
    else:
        #Si no existe la duración en la sección, appendeamos None
        horas_y_minutos.append(None)

            
    #Ahora, el contenido de horas_y_minutos es [(2, 2), (1, 30), (1, 47), (1, 44), ('NULL)]. Vamos a pasarlo a minutos.
    #Iteramos horas_y_minutos
    for tiempo in horas_y_minutos:
        #Si es 'NULL', appendeamos None
        if tiempo == None:
            lista_duraciones.append(None)
        #Se puede dar el caso de que sólo haya un dígito, por ejemplo 2 (2 horas exactas). De ser así, multiplicamos las horas por 60.
        elif len(tiempo)==1:
            lista_duraciones.append(tiempo[0]*60)
        else:
            #Si hay más de un dígito, multiplicamos el primero por 60 y le sumamos los minutos.
            lista_duraciones.append((tiempo[0]*60) + (tiempo[1]))
            


#Cerramos el navegador      
driver.close()   

100%|██████████| 1000/1000 [1:53:31<00:00,  6.81s/it] 


6. Extraemos datos de Rotten Tomatoes

In [55]:
#1. Entramos en la website the 'rotten tomatoes' y aceptamos cookies
driver=webdriver.Chrome()
driver.get('https://www.rottentomatoes.com/')
driver.maximize_window()
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(3)

#2. Iteramos el zip para extraer su puntuación
for titulo, año in tqdm(zip_pelis_cortos):

    #Intentamos buscar por el título; si existe entra en el primer resultado; de lo contrario, appenda un mensaje de 'NULL' (ponemos NULL para que las futuras consultas en SQL sean más prácticas)
    try: #Buscamos la peli por título y año en la barra del buscador
        driver.find_element('css selector', '#header-main > search-algolia > search-algolia-controls > input').send_keys(f'{titulo} {año}', Keys.ENTER)
        sleep(3) #Hacemos click en el primer resultado
        driver.find_element('css selector', '#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child(1) > a:nth-child(2)').click()
        sleep(3)
        try: #busca el tomatómetro y, si existe, lo appendea en la lista_punt_tomato
            lista_puntu_tomato.append(driver.find_element('css selector', '#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text').text)
        except: #si no tiene tomatómetro, appendea 'NULL'
            lista_puntu_tomato.append(None)
        sleep(3)
    except: #si la peli no existe devuelve None
        lista_puntu_tomato.append(None)

#Cerramos el navegador      
driver.close() 

100%|██████████| 1000/1000 [5:16:00<00:00, 18.96s/it] 


7. Unimos todas las listas en un zip y construimos el DataFrame

In [109]:
zip_fase2=zip(lista_id, lista_puntu_IMDB, lista_puntu_tomato, lista_dires, lista_guionista, lista_argumentos, lista_duraciones, lista_nombre_pelis)
lista_tuplas_fase2=list(zip_fase2)
len(lista_tuplas_fase2)

1000

In [110]:
df_fase2=pd.DataFrame(lista_tuplas_fase2)
df_fase2.columns=['Id_pelicula', 'Puntuacion_imdb', 'Puntuacion_rotten', 'Directores', 'Guionistas', 'Argumento', 'Duracion', 'Nombre_pelicula']

In [111]:
df_fase2.to_csv("Fase2.csv", index=False)